CNN example as previous examples, this time using purely TensorFlow, without Keras overlay

Troubles with spliting the data info batched with *TypeError* "Unhashable"

In [38]:
import tensorflow as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
from numpy.testing import assert_equal
from PIL import Image


In [21]:
#load data
path = '/Users/oziomek/licencjat/kostki/zdjecia/'
listing = os.listdir(path)
print(listing)
num_samples = 0
for folder in os.listdir(path):
    for i in range(1, 7):
        num_samples += len(os.listdir(path + folder + '/' + str(i)))
assert_equal(100800, num_samples)
print(num_samples)

data = np.array(
    [np.array(Image.open(path + folder + '/' + str(i) + '/' + file)).flatten() 
         for folder in os.listdir(path)
             for i in range(1, 7) 
                 for file in os.listdir(path + folder + '/' + str(i))], 'f')
print(data.shape)

['navyOnWhite', 'greenOnGreen', 'navyOnBlue', 'blackOnBlack', 'whiteOnBlue', 'blackOnRed', 'redOnRed', 'stainOnWhite', 'redOnRed_white', 'whiteOnBlack', 'hardLight', 'whiteOnRed', 'greenOnWhite', 'woodOnRed', 'whiteOnRed_distance']
100800
(100800, 4096)


In [39]:
def cnn_model(features, labels, mode):
    
    #input
    input_layer = tf.reshape(features['x'], [-1, 64, 64, 1])
    
    #convolutional1
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding='same',
        activation='relu')
    
    #pooling1
    pool1 = tf.layers.max_pooling2d(
        inputs=conv1, 
        pool_size=[2, 2], 
        strides=1)
    
    #convolutional2
    conv1 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding='same',
        activation='relu')
    
    #dropout1
    dropout1 = tf.layers.dropout(
        inputs=conv2, 
        rate=0.2,
        training=mode == tf.estimator.ModeKeys.TRAIN)
    
    #flatten1
    flat1 = tf.reshape(
        dropout1, 
        [-1, 32*32*64])
    
    #dense1
    dense1 = tf.layers.dense(
        inputs=flat1, 
        units=1024, 
        activation='relu')
    
    #dropout2
    dropout2 = tf.layers.dropout(
        inputs=dense1, 
        rate=0.2,
        training=mode == tf.estimator.ModeKeys.TRAIN)
    
    #dense2
    dense2 = tf.layers.dense(
        inputs=dropout2, 
        units=256, 
        activation='relu')
    
    #dropout2
    dropout3 = tf.layers.dropout(
        inputs=dense2, 
        rate=0.2,
        training=mode == tf.estimator.ModeKeys.TRAIN)
    
    #dense3
    dense3 = tf.layers.dense(
        inputs=dropout3, 
        units=64, 
        activation='relu')
    
    labeled = tf.layers.dense(
        inputs=dense3,
        units=6)
    
    predictions = {
        "classes": tf.argmax(input=labeled, axis=1),
        "probabilities": tf.nn.softmax(labeled, name='softmax_tensor')
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # for train and eval mode
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    # for train mode
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(
            learning_rate=0.001,
            beta1=0.9,
            beta2=0.999,
            epsilon=1e-08,
            use_locking=False,
            name='Adam')
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    # for eval mode
    eval_metrics_ops = {
        'accuracy': tf.metrics.accuracy(
            labels=labels,
            predictions=predictions['classes'])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metrics_ops=eval_metrics_ops)
    

In [48]:
def main(arg):
    
    labels = np.ones((num_samples, ), dtype=int)
    counter = 0
    for folder in os.listdir(path):
        for i in range(1, 7):
            samples = len(os.listdir(path + folder + '/' + str(i)))
            labels[counter:counter+samples] = i-1
            counter += samples
            
    rows, cols = 64, 64
    labels_values = 6
    train_data = [data, labels]
    import random
    for i in range(3):
        random_image = random.randint(0, 100800)
        img=data[random_image].reshape(rows, cols)
        plt.imshow(img)
        plt.show()
        print("There are {} dots, number is {}".format(labels[random_image] + 1, random_image))
        
    (X, y) = (train_data[0], train_data[1])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)
    
    X_train = X_train.reshape(X_train.shape[0], rows, cols, 1).astype('float32')
    X_test = X_test.reshape(X_test.shape[0], rows, cols, 1).astype('float32')

    # normalize to 0-1
    X_train /= 255
    X_test /= 255

    print(X_train.shape, X_test.shape)
    
    y_train = tf.one_hot(y_train, 6)
    y_test = tf.one_hot(y_test, 6)
    print(y_train.shape, y_test.shape)
    
    # Create the Estimator
    dice_classifier = tf.estimator.Estimator(
        model_fn=cnn_model, model_dir="/dice_convnet_model")

    # Set up logging for predictions
    # Log the values in the "Softmax" tensor with label "probabilities"
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
        tensors=tensors_to_log, every_n_iter=5)
    
    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_train},
        y=y_train,
        batch_size=512,
        num_epochs=25,
        shuffle=True)
    dice_classifier.train(
        input_fn=train_input_fn,
        steps=20000,
        hooks=[logging_hook])
    
    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_test},
        y=y_test,
        num_epochs=1,
        shuffle=False)
    eval_results = dice_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

In [ ]:
if __name__ == "__main__":
  tf.app.run()